In [18]:
#Biblioteket
import numpy as np

In [19]:
#Enklar deklaratioer
#m= 1000     #(kg) är massan
mu=3        #Friktionskoffecient 2=<mu<=4, vi valde medelvärdet
g= 9.81     #(m/s^2) tyngdfaktor

Cd= 0.3     #luftmotståndskoeffecient ENHETSLÖS!
Af= 1.63    #Area-front ENHETSLÖS!
p_air=1.2   #(kg/m^3)   luftdensitet 
#v_veh=150   #(m/s) Hastigheten som vi skall VARIERAR ENLIGT ÖNSKEMÅL

C_rr= 0.03   #rullmotståndskoffecient 
#alpha= 0    #(grader) lutningen i grander VARIERAR ENLIGT ÖNSKEMÅL

#a_veh=10    #(m/s^2) bilens acceleration vid nån-tidspunkt VARIERAR ENLIGT ÖNSKEMÅL

In [20]:
#Implementering av ekvationerna som vi har från In 2 del (TEKNISKA KRAV)

#a_max= mu * g       #(m/s^2) Max acceleration innan slieringen
#print("a_max=",a_max)

#F_veh= m * a_max    #(N) Maximala kraften vi kan ha innan däck slirar
#print("F_veh=", F_veh)        

def F_aero(v_veh):  #(N) Luftmotstånds-kraft
    F_aero = (Cd * Af * p_air * (v_veh**2)) / 2 
    return F_aero
#print("F_aero(v_veh)=",F_aero(v_veh))

def F_roll(alpha, v_veh, m):   #(N) Friktionskraft
    F_roll = 0
    if v_veh == 0:
        F_roll = 0
    else:
        F_roll = C_rr * m * g * np.cos(alpha) 
    return F_roll
#print("F_roll(alpha, v_veh)=",F_roll(alpha, v_veh, m))

def F_grad(alpha, m):     #(N) Den vertikala gradientens kraftkomposant 
    F_grad= m * g * np.sin(alpha)
    return F_grad
#print("F_grad(alpha)=",F_grad(alpha,m))

#F_net= m * a_veh        #(N) Den resulterande kraften 
#print("F_net=",F_net)

#F_net=F_trac - F_roll - F_grad --> F_trac= F_aero + F_roll + F_grad + F_net 
#F_trac = F_aero(v_veh) + F_roll(alpha, v_veh) + F_grad(alpha) + F_net    #(N) Den drivandekraften från motor till hjulen
#print("F_trac=",F_trac)

In [21]:
#Implementering av ekvationerna som vi har från In 2 del (FYSIKALISK MODELL)

#----------------------Enkla Deklarationer--------------------------------------

#=======================Beräkningen för lamda==========================
#(U_nom -I_nom * Ra)/w_r = lamda
#U_nom= 1873.02 (V), I_nom= 467.325 (A), Ra = 0.090675 (Ohm), w_r= 184.8375 (rad/s)
#=====> lamda = 9.90407956
#=======================================================================
lamda= 9.90407956                  #Konstant ENHETSLÖS!
Ra=0.039 * 2.325            #(Ohm) Resistansten 
r_hjul= 0.46482             #(m) 36.6 (inches) / 2 *0.0254 = 0.46482 (m)
T_f= 15.905 * 2.325                      #(Nm) Friktionsförluster i motor VI BORTSER FRÅN DE 
T_L= 0                      #(Nm) Lastmoment    VARIERAR ENLIGT ÖNSKEMÅL
n= 0.982                    #Verkningsgrad ENHETSLÖS!
i= 5                        #Utväxlingen ENHETSLÖS! (Vi valde planet Växel)

#-----------------------Ekvationer----------------------------------------------
def T_e(ia):            #(Nm) Motorvridmoment
    T_e= ia * lamda
    return T_e
#print("T_e(F_trac)=", T_e(ia)) 

def w_r(v_veh):             #(rad/s) varvtal för elmotorn
    w_r= i * (v_veh/r_hjul)     
    return w_r
#print("w_r(v_veh)=",w_r(v_veh))

def ia(F_trac, v_veh):      #(A) Stationär ankarsström 
    ia= (1/lamda) * T_e(F_trac)
    return ia
#print("ia(F_trac, v_veh)=",ia(F_trac, v_veh))

def U_t(F_trac, v_veh):     #(V) Stationär spänning 
    U_t= Ra * (1/lamda) * T_e(F_trac) + lamda * w_r(v_veh)
    return U_t
#print("U_t(F_trac, v_veh)=",U_t(F_trac, v_veh))



In [22]:
#=====================================================================
#-----------------------Simulering---------------------------------------
#=====================================================================
#-----------------Simulering funktion------------------------------------------
x = 1                           #Variabel för att bestämma spänning- eller ström styrd
U = [2679, x]                   #(v) 
I = [669, 1-x]                  #(A)
I_EM = 0                        #(A) 
def sim(m,alpha, U, I):         #simulerig med insignalerna m, alpha, U, I 
    if U[1]==True:              #konstant spänning / dvs spänningstyrd
        simtyp = 1
        U=U[0]                  #U=2679 (V)
    else: 
        simtyp = 0              #Konstant ström / dvs strömstyrd
        I_Em=I[0]               #I= 669 (A)

    N = 1000                    #Antal iterrationer 
    dt= 0.001                   #(s) Tidsteg (start fråm 1ms)
    t_tot = N *dt               #(s) Totala simuleringstid 
#------------------Skapar vektorer för att lagra in resultat-------------
    t_vec = np.zeros(N)         #tid            ex.[0,0,....,0]
    v_vec = np.zeros(N)         #hastighet      ex.[0,0,....,0]
    a_vec = np.zeros(N)         #acceleration   ex.[0,0,....,0]
    ia_vec = np.zeros(N)        #ankarstörm     ex.[0,0,....,0]
    U_vec = np.zeros(N)         #spänning       ex.[0,0,....,0]
    s_vec = np.zeros(N)         #sträcka        ex.[0,0,....,0]

    for k in range(0,N):        #Från 0 till 999 (1000 iter.)
        t_vec[k]= k * dt                                #Sparar tiden för tillstånd k 
        #Mekaniska krafter från omvärden som verkar på fordonet
        F_aero_1 = F_aero(v_vec[k])                     #Räknar luftmotsåndet i tillstånd k för hastigheten i det tillståndet
        F_roll_1 = F_roll(alpha,v_vec[k], m)            #Räknar friktion i tillstånd k för hastigheten i det tillståndet
        F_grad_1 = F_grad(alpha,m)                      #Räknar lutning i tillstånd k för hastigheten i det tillståndet
        F_motstånd= F_aero_1 + F_roll_1 + F_grad_1      #Totala motståndet som verkar på fordonet
        #Mekaniksa kraften vi får från motorn
        if simtyp==1:                                   #Fallet spänningstyrd
            I_EM = (U - lamda * w_r(v_vec[k]))/Ra 
            ia_vec[k]=I_EM                              #Sparar värden för strömmen för varje tillstånd
        else:                                           #Strömstyrd
            U_vec[k]=I_EM * Ra + lamda * w_r(v_vec[k])  #Sparar värden för späningen för varje tillstånd
        T_hjul = n*i*(T_e(I_EM)-T_f)                    #(Nm) Räknar hjulensvridmoment beroende av strömmen
        F_trac = T_hjul/r_hjul                          #(N) Dragkraften vid hjulen    
        F_net = F_trac - F_motstånd                     #(N) Nettokraften som får fordonet att röra sig
        a = F_net/m                                     #(m/s^2) Accelerationen
        a_vec[k]=a                                      #Sparar accel i varje tillstånd
        v_vec[k+1]=v_vec[k]+ (a * dt)                   #(Eulers-metod) Sparar hastigheten i nästa tillstånd
        if k==N-1:                                      #Speciall fall
            #ia_vec[N]=(U - lamda * w_r(v_vec[N]))/Ra 
            #U_vec[N] =I_EM * Ra + lamda * w_r(v_vec[N]) 
            T_hjul = n*i*(T_e(I_EM)-T_f)                    #(Nm) Räknar hjulensvridmoment beroende av strömmen
            F_trac = T_hjul/r_hjul                          #(N) Dragkraften vid hjulen    
            F_net = F_trac - F_motstånd                     #(N) Nettokraften som får fordonet att röra sig
            a = F_net/m                                     #(m/s^2) Accelerationen
            a_vec[N]=a
        #S = v * t + (a*t^2)/2
        s_vec[k+1]=s_vec[k]+ (v_vec[k] * k * dt)+(a * ((k*dt)**2))/2 #Sparar sträckan i nästa tillstånd

        
        




